In [58]:
#import library
import pandas as pd
import numpy as np
import nltk
import Sastrawi
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from nltk.tag import CRFTagger
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

In [59]:
#membaca data
raw_data = pd.read_csv("dataset/train_set.csv", delimiter=",", encoding="Latin-1")
raw_data.head()

,id,sentimen,tweet
0,1,1,oks kak semangat ya kalian kalian
1,2,0,sekarang harus kaya orang bodoh lagi bodoh sangat
2,3,1,"Begitu diumumkan lulus 100%, mereka semua suju..."
3,4,0,[USERNAME] [USERNAME] Katanya Bapak Reformasi ...
4,5,0,macet macetan perut kosong akhirnya mampir dah...


In [60]:
# read stopwords
stopwords = pd.read_csv("dataset/stopwords.csv")
stopwords.head()

,ada
0,adalah
1,adanya
2,adapun
3,agak
4,agaknya


In [61]:
def normalisasi(tweet):
    normal_tw = tweet.lower() #lowercase
    normal_tw = re.sub('\s+', ' ', normal_tw) # remove extra space
    normal_tw = normal_tw.strip() #trim depan belakang
    normal_tw = re.sub(r'[^\w\s]','',normal_tw) #buang punctuation
    normal_regex = re.compile(r"(.)\1{1,}", re.IGNORECASE) #regex huruf yang berulang kaya haiiii (untuk fitur unigram)
    normal_tw = normal_regex.sub(r"\1\1", normal_tw) #buang huruf yang berulang
    return normal_tw

In [62]:
def remove_stopwords(tweet):
    special_list = ['username', 'url', 'sensitive-no']
    token = nltk.word_tokenize(tweet)
    token_afterremoval = []
    for k in token:
        if k not in stopwords and k not in special_list:
            token_afterremoval.append(k)
    str_clean = ' '.join(token_afterremoval)
    return str_clean

In [63]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
def stemming(tweet):
    token = nltk.word_tokenize(tweet)
    stem_kalimat = []
    for k in token:
       factory = StemmerFactory()
       stemmer = factory.create_stemmer()
       stem_kata = stemmer.stem(tweet)
       stem_kalimat.append(stem_kata)   
    stem_kalimat_str = ' '.join(stem_kalimat)

In [70]:
def preprocessing(list_tweet):
    # id tweet incremental, jadi simpen aja list berurut
    tweet_clean = []
    for tw in list_tweet:
        normal_tweet = normalisasi(tw)
        nosw_tweet = remove_stopwords(normal_tweet)
        
        stem_tweet = stemming(nosw_tweet)
        tweet_clean.append(nosw_tweet)
    return tweet_clean

In [65]:
def ekstraksiBoW(tweet):
    unigram = CountVectorizer(ngram_range=(1,1), max_features=2000)
    unigram_matrix = unigram.fit_transform(np.array(tweet)).todense()
    nama_fitur = unigram.get_feature_names()
    return unigram_matrix, nama_fitur

In [66]:
def ekstraksiSentimen(list_tweet):
    pos = pd.read_csv('dataset/positif_vania.txt', header=None, names=['pos'])
    list_pos = pos['pos'].tolist()
    neg = pd.read_csv('dataset/negatif_vania.txt', header=None, names=['neg'])
    list_neg = neg['neg'].tolist()

    fitur_sentimen_all = []
    for tweet in list_tweet:
        ##inisiasi value
        emosi = ["positif", "negatif"]
        value = [0,0]
        emosi_value = {}
        for i in range(len(emosi)):
            emosi_value[emosi[i]] = value[i]
    
        list_kata = tweet.split()
        for k in list_kata:
            if k in list_pos:
                emosi_value["positif"] += 1
            if k in list_neg:
                emosi_value["negatif"] += 1


        fitur_sentimen_perkalimat = list(emosi_value.values())
        fitur_sentimen_all.append(fitur_sentimen_perkalimat)

    return fitur_sentimen_all

In [67]:
def ekstraksiPOS(list_tweet):
    ct = CRFTagger()
    ct.set_model_file("dataset/all_indo_man_tag_corpus_model.crf.tagger")
    pos_feat_list = []
    count_tag = []
    for tweet in list_tweet:
        token = nltk.word_tokenize(tweet)
        tag = ct.tag_sents([token])
        flat_tag = [item for sublist in tag for item in sublist]
        pos_count = Counter([j for i,j in flat_tag])
        pos_feat = (pos_count['JJ'], pos_count['NEG'])
        pos_feat_list.append(pos_feat)

    return pos_feat_list

# Simpen state sampe stemming dulu

In [68]:
def create_state(data_tweets):
    # create preprocessing state
    clean_data = raw_data.copy(deep=True)
    clean_data['tweet'] = data_tweets
    with open('dataset/preprocess_state.csv', 'w') as state_file:
        state_file.write(clean_data.to_csv(index=False))
    return clean_data.head()

In [71]:
raw_tweet = raw_data['tweet']
clean_tweets = preprocessing(raw_tweet)
create_state(clean_tweets)

,id,sentimen,tweet
0,1,1,oks kak semangat ya kalian kalian
1,2,0,sekarang harus kaya orang bodoh lagi bodoh sangat
2,3,1,begitu diumumkan lulus 100 mereka semua sujud ...
3,4,0,katanya bapak reformasi dan demokrasi di neger...
4,5,0,macet macetan perut kosong akhirnya mampir dah...


In [1]:
#Membaca data menggunakan pandas
#import library
import pandas as pd
import numpy as np
import nltk
import Sastrawi
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from nltk.tag import CRFTagger
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
raw_data = pd.read_csv('dataset/preprocess_state.csv', encoding = "Latin-1")
print(raw_data.shape)
raw_data.head()

(3462, 3)


,id,sentimen,tweet
0,1,1,oks kak semangat ya kalian kalian
1,2,0,sekarang harus kaya orang bodoh lagi bodoh sangat
2,3,1,begitu diumumkan lulus 100 mereka semua sujud ...
3,4,0,katanya bapak reformasi dan demokrasi di neger...
4,5,0,macet macetan perut kosong akhirnya mampir dah...
